# Reinforcement Learning: an introductory example

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Reinforcement-Learning:-an-introductory-example" data-toc-modified-id="Reinforcement-Learning:-an-introductory-example-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Reinforcement Learning: an introductory example</a></span><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Introducing-the-example-network" data-toc-modified-id="Introducing-the-example-network-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Introducing the example network</a></span><ul class="toc-item"><li><span><a href="#Import-and-prepare-data" data-toc-modified-id="Import-and-prepare-data-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Import and prepare data</a></span></li><li><span><a href="#Create-graph" data-toc-modified-id="Create-graph-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Create graph</a></span></li></ul></li></ul></li></ul></div>

## Libraries

In [ ]:
%matplotlib inline
# import libraries
import networkx as nx
import matplotlib
from RL.input import input_data

In [ ]:
# load input class
grid_data = input_data.InputData()

## Introducing the example network   

In this notebook we look at a small neighbourhood with one netstation (MSR), 5 houses and a couple of streets connecting the houses with the MSR. First the neighbourhood will be drawn as a network graph with nodes and edges.

The goal is develop an algorithm that thinks like an engineer and is capable to design an electrical grid that meets certain constraints.

### Import and prepare data  

Here the grid data is imported and the nodes and edges are converted into a networkx graph. The regular network nodes are plotted in red and the MSR node is shown in blue.

In [ ]:
# import csv files and prepare the grid data, i.e. create node and edge list and a positional
# dictionary of nodes
grid_data.import_csv_as_df()
grid_data.prepare_grid_data()

In [ ]:
grid_data.df_edges.head()

In [ ]:
grid_data.df_nodes.head()

### Create graph

Define a networkx graph object and add nodes and edges to the graph.

In [ ]:
grid_data.update_networkx_graph()

The data has been prepared. Draw the grid of this easy example neighbourhood to see what it looks like.

In [ ]:
nx.draw_networkx_nodes(grid_data.network_graph, 
                       pos=grid_data.nodes_pos, 
                       nodelist=grid_data.node_list, node_color='b', node_size=100)
nx.draw_networkx_nodes(grid_data.network_graph, 
                       pos=grid_data.nodes_pos, 
                       nodelist=grid_data.node_list_msr, node_color='r', node_size=150)
nx.draw_networkx_nodes(grid_data.network_graph, 
                       pos=grid_data.household_nodes_pos, 
                       nodelist=grid_data.node_list_households, node_color='g', node_size=50)
nx.draw_networkx_edges(grid_data.network_graph, 
                       pos=grid_data.nodes_pos,
                       edgelist=grid_data.edge_tuples)